# Day 9: ⏪ Undo Buttons for your Agents

> Building an "Edit Message" or "Regenerate" feature shouldn't require complex database migrations. In the ADK, time travel is built-in.

## 今日目标

学习 ADK 的 **Session Rewind** 功能，实现：
- 编辑消息 (Edit Message)
- 重新生成 (Regenerate)
- 撤销操作 (Undo)

## 官方资源

- [ADK Sessions Rewind](https://google.github.io/adk-docs/sessions/rewind/)
- [ADK Runtime Resume](https://google.github.io/adk-docs/runtime/resume/)
- [官方示例代码](https://github.com/google/adk-python/blob/main/contributing/samples/rewind_session/main.py)

## 1. 环境准备

In [ ]:
import sys
sys.path.insert(0, '..')

from shared.config import load_env
load_env()

In [ ]:
from google.adk import Agent
from google.adk.tools.tool_context import ToolContext
from google.adk.agents.run_config import RunConfig
from google.adk.runners import InMemoryRunner
from google.genai import types

## 2. 定义工具函数

我们创建一个可以管理状态和文件的 Agent，用来演示回溯功能。

In [ ]:
async def update_state(tool_context: ToolContext, key: str, value: str) -> dict:
    """更新状态值"""
    tool_context.state[key] = value
    return {"status": f"已更新 '{key}' 为 '{value}'"}


async def load_state(tool_context: ToolContext, key: str) -> dict:
    """读取状态值"""
    return {key: tool_context.state.get(key, "未找到")}


async def save_artifact(tool_context: ToolContext, filename: str, content: str) -> dict:
    """保存文件内容"""
    artifact_bytes = content.encode("utf-8")
    artifact_part = types.Part(
        inline_data=types.Blob(mime_type="text/plain", data=artifact_bytes)
    )
    version = await tool_context.save_artifact(filename, artifact_part)
    return {"status": "成功", "filename": filename, "version": version}


async def load_artifact(tool_context: ToolContext, filename: str) -> dict:
    """读取文件内容"""
    artifact = await tool_context.load_artifact(filename)
    if not artifact:
        return {"error": f"文件 '{filename}' 未找到"}
    content = artifact.inline_data.data.decode("utf-8")
    return {"filename": filename, "content": content}

## 3. 创建 Agent

In [ ]:
state_agent = Agent(
    name="state_agent",
    model="gemini-2.0-flash",
    instruction="""你是一个状态和文件管理 Agent。

你可以：
- 更新状态值 (update_state)
- 读取状态值 (load_state)
- 保存文件 (save_artifact)
- 读取文件 (load_artifact)

根据用户的请求使用相应的工具。用中文回复。""",
    tools=[
        update_state,
        load_state,
        save_artifact,
        load_artifact,
    ],
)

print("✅ Agent 创建成功")

## 4. 创建辅助函数

In [ ]:
APP_NAME = "rewind_demo"
USER_ID = "demo_user"


async def call_agent(runner, session_id: str, prompt: str):
    """调用 Agent 并打印结果"""
    print(f"\n👤 用户: {prompt}")
    content = types.Content(
        role="user", parts=[types.Part.from_text(text=prompt)]
    )
    events = []
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session_id,
        new_message=content,
        run_config=RunConfig(),
    ):
        events.append(event)
        if event.content and event.author and event.author != "user":
            for part in event.content.parts:
                if part.text:
                    print(f"  🤖 Agent: {part.text}")
                elif part.function_call:
                    print(f"    🛠️ 工具调用: {part.function_call.name}({part.function_call.args})")
                elif part.function_response:
                    print(f"    📦 工具响应: {part.function_response.response}")
    return events

## 5. Session Rewind 演示

### 5.1 创建 Runner 和 Session

In [ ]:
# 创建 Runner
runner = InMemoryRunner(
    agent=state_agent,
    app_name=APP_NAME,
)

# 创建会话
session = await runner.session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID
)
print(f"📝 创建会话: {session.id}")

### 5.2 步骤 1: 初始化状态

In [ ]:
# 设置初始状态
await call_agent(runner, session.id, "设置状态 color 为 red")

In [ ]:
# 保存初始文件
await call_agent(runner, session.id, "保存文件 note.txt 内容为 version1")

### 5.3 步骤 2: 确认当前状态

In [ ]:
await call_agent(runner, session.id, "查询状态 color 的值")

In [ ]:
await call_agent(runner, session.id, "读取文件 note.txt")

### 5.4 步骤 3: 修改状态 ⚠️ 这是回溯点

In [ ]:
# 这次修改是我们后面要撤销的
events_update = await call_agent(runner, session.id, "更新状态 color 为 blue")

# 记录这个操作的 invocation_id，用于后续回溯
rewind_invocation_id = events_update[0].invocation_id
print(f"\n📌 记录回溯点 invocation_id: {rewind_invocation_id}")

In [ ]:
# 再保存新版本的文件
await call_agent(runner, session.id, "保存文件 note.txt 内容为 version2")

### 5.5 步骤 4: 确认修改后的状态

In [ ]:
await call_agent(runner, session.id, "查询状态 color 的值")

In [ ]:
await call_agent(runner, session.id, "读取文件 note.txt")

### 5.6 步骤 5: ⏪ 执行 REWIND！

现在我们要回溯到 "color = blue" 那次修改之前的状态。

In [ ]:
print(f"⏪ 回溯到 invocation_id: {rewind_invocation_id} 之前...")

await runner.rewind_async(
    user_id=USER_ID,
    session_id=session.id,
    rewind_before_invocation_id=rewind_invocation_id,
)

print("✅ 回溯完成！")

### 5.7 步骤 6: 验证回溯后的状态

回溯后，状态应该恢复到：
- color = **red** (而不是 blue)
- note.txt = **version1** (而不是 version2)

In [ ]:
await call_agent(runner, session.id, "查询状态 color 的值")

In [ ]:
await call_agent(runner, session.id, "读取文件 note.txt")

## 6. 总结

### 🔑 核心要点

1. **`runner.rewind_async()`** - 回溯会话到指定的 invocation 之前
2. **`rewind_before_invocation_id`** - 指定回溯点的 ID
3. **完整恢复** - 回溯会同时恢复 state（状态）和 artifacts（文件）

### 💡 应用场景

| 功能 | 实现方式 |
|------|----------|
| 编辑消息 | 回溯到该消息之前，然后发送新消息 |
| 重新生成 | 回溯到上一个用户消息之前，然后重新运行 |
| 撤销操作 | 回溯到操作之前 |
| 分支对话 | 回溯到某个点，然后走不同的路径 |

### 📚 参考链接

- [ADK Sessions Rewind 文档](https://google.github.io/adk-docs/sessions/rewind/)
- [ADK Runtime Resume 文档](https://google.github.io/adk-docs/runtime/resume/)
- [官方示例代码](https://github.com/google/adk-python/blob/main/contributing/samples/rewind_session/main.py)